In [1]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn

In [2]:
def pre_processing(sentence):
    return remove_stopwords(tokenize_sentence(remove_punctuation(sentence)))

# Remove punctuation from a list of words
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence)

# Remove stopwords from a list of words
def remove_stopwords(words_list):
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    return [value.lower() for value in words_list if value.lower() not in stopwords_list]

# Tokenize the input sentence and also lemmatize its words
def tokenize_sentence(sentence):
    words_list = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        if (tag[1][:2] == "NN"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.NOUN))
        elif (tag[1][:2] == "VB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.VERB))
        elif (tag[1][:2] == "RB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADV))
        elif (tag[1][:2] == "JJ"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADJ))
    return words_list

In [3]:
#ogni articolo del file verrà rappresentato come una lista
#di parole pre-processate
from gensim import corpora, models

FILE = 'italian_cuisine_2.txt'  # il file contiene un insieme di documenti divisi per paragrafi
TOPIC_NUMBER = 10
WORDS_FOR_TOPIC = 5

with open(FILE, encoding='utf-8') as file:
    # lista di liste. Ogn lista rappresenta un documento ed è formata da una sequenza di parole
    list_document_words = []

    for line in file:
        if "<doc" in line:  # nuovo documento
            document_words = []  # lista di parole che faranno parte del documento
            while True:
                next_line = file.readline()  # read next line

                # rimozione tag paragrafi
                new_next_line = next_line.replace("<p> ", "").replace("</p>\n", "").replace("/p", "")

                if "</doc>" in next_line:  # documento terminato
                    break
                # rimuovo punteggiatura, tokenizzo e rimuovo stopwords
                sentence_words = pre_processing(new_next_line)
                document_words.extend(sentence_words)
            list_document_words.append(document_words)
    file.close()

    print("Numero di documenti: ", len(list_document_words))

    # crea un dizionario che incapsula il mapping tra le parole normalizzate
    # e i loro indentificatori (interi)
    dictionary_LDA = corpora.Dictionary(list_document_words)

    # elimina tutti i termini che NON appaiono in almeno 3 documenti
    # elimina tutti i termini che appaiono in oltre il 50% dei documenti (vedere doc. ufficiale di filter_extremes)
    dictionary_LDA.filter_extremes(no_below=3)

    # converte ogni documento (sottoforma di lista di parole)
    # in un bag of words. Corpus sarà una lista di bag of words ognuna
    # delle quali rappresenta un documento
    corpus = [dictionary_LDA.doc2bow(document_words) for document_words in list_document_words]

    lda_model = models.LdaModel(corpus, num_topics=TOPIC_NUMBER, \
                                id2word=dictionary_LDA, \
                                passes=4, alpha=[0.01] * TOPIC_NUMBER, \
                                eta=[0.01] * len(dictionary_LDA.keys()))

    for i, topic in lda_model.show_topics(formatted=True, num_topics=TOPIC_NUMBER, num_words=WORDS_FOR_TOPIC):
        print(str(i) + ": " + topic)
        print()

    # distribuzione di probabilità dei topic nel documento 5
    print(lda_model[corpus[5]])  # corpus[0] means the first document.

C:\Users\thewp\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Numero di documenti:  310
0: 0.039*"knife" + 0.017*"chef" + 0.014*"blade" + 0.011*"kitchen" + 0.010*"chefs"

1: 0.053*"cake" + 0.014*"pan" + 0.010*"bake" + 0.009*"flour" + 0.008*"italian"

2: 0.028*"subscribe" + 0.025*"email" + 0.023*"newsletter" + 0.019*"feature" + 0.019*"sign"

3: 0.010*"rice" + 0.010*"sushi" + 0.010*"cuisine" + 0.009*"dish" + 0.007*"fish"

4: 0.022*"chicken" + 0.018*"italian" + 0.013*"wedding" + 0.008*"salad" + 0.008*"bride"

5: 0.011*"italian" + 0.011*"cuisine" + 0.009*"dish" + 0.008*"wok" + 0.008*"pan"

6: 0.022*"italian" + 0.019*"blog" + 0.012*"chef" + 0.008*"authority" + 0.008*"post"

7: 0.022*"pork" + 0.022*"sauce" + 0.015*"sausage" + 0.015*"italian" + 0.010*"meat"

8: 0.071*"chop" + 0.059*"pork" + 0.012*"chops" + 0.010*"side" + 0.009*"pan"

9: 0.014*"italian" + 0.012*"chop" + 0.010*"pan" + 0.009*"pork" + 0.008*"oven"

[(4, 0.6738991), (9, 0.32528493)]


In [ ]:
conda install -c conda-forge python-levenshtein

In [ ]:
pip install python-Levenshtein